# Official public work with private tools

This report is dedicated to the analysis of the data on Hilary Clinton'email. The idea of this project is to understand the email controversy arising from use of private email server for official communiactions during her tenure as Secretary of State hit the headline of the media in 2015.

## Data extraction and exploration

In this section, we extract the data from Kaggle. We use the sql file since there are four csv files. We use the package sqlite to extract the data and also all the already used package for the applied data analysis courses.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time
import nltk
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

In [2]:
data_folder = "../data/"

In [3]:
#Data extraction from the csv-files
emails_raw = pd.read_csv(data_folder + 'Emails.csv')
persons = pd.read_csv(data_folder + 'Persons.csv')
aliases = pd.read_csv(data_folder + 'Aliases.csv')
email_receivers = pd.read_csv(data_folder + 'EmailReceivers.csv')

In [4]:
print("emails:", emails_raw.shape)
print("persons:", persons.shape)
print("aliases:", persons.shape)
print("email receivers:", email_receivers.shape)

emails: (7945, 22)
persons: (513, 2)
aliases: (513, 2)
email receivers: (9306, 3)


The data is available in two formats: csv and sql. This means that the elements are somehow related. We will see later that the entity relationship diagram is not as it should have been done.

For the moment, let's discover what we have. Extraction was performed according to some fields. Let us check whether or not we can rely on it.

In [ ]:
persons.head()

In [ ]:
persons.dtypes

In [ ]:
aliases.head()

In [ ]:
aliases.dtypes

It is worth to mention one person can have multiple aliases. However, each alias points to a single person.

In [ ]:
aliases.PersonId.is_unique

In [ ]:
aliases.Alias.is_unique

In [ ]:
email_receivers.head()

In [ ]:
email_receivers.dtypes

In [ ]:
emails_raw.head()

In [ ]:
aliases[aliases.Alias.str.strip() == ';H']

We cannot perform a direct mapping from the fields of `emails` to the fields of `aliases`.

This table contains 22 columns and the pandas does not display everything.

In [ ]:
emails_raw.dtypes

Unfortunately, we will have to perform some actions to get the identities of the 'from', 'to' and 'cc'. Actually, we have the value of the 'to' from the table `emails_receiver` and it points towards the `PersonId` field. However, the fields 'from' and 'cc' point towards the `aliases` (not even its identifiers!).

In [ ]:
emails_raw.DocNumber.is_unique

In [ ]:
emails_raw.MetadataCaseNumber.unique()

In [ ]:
emails_raw.MetadataDocumentClass.unique()

In [ ]:
emails_raw.ExtractedReleaseInPartOrFull.unique()

The meaning of these fields are obscure. It is time to cast what we can.

In [5]:
emails_raw.MetadataDateSent = emails_raw.MetadataDateSent.map(pd.to_datetime)
emails_raw.MetadataDateReleased = emails_raw.MetadataDateReleased.map(pd.to_datetime)
emails_raw.ExtractedDateReleased = emails_raw.ExtractedDateReleased.map(pd.to_datetime)
emails_raw.ExtractedReleaseInPartOrFull = emails_raw.ExtractedReleaseInPartOrFull.astype('category') # 3 enumerations
emails_raw.MetadataCaseNumber = emails_raw.MetadataCaseNumber.astype('category') # 5 enumerations
emails_raw.MetadataDocumentClass = emails_raw.MetadataDocumentClass.astype('category') # 4 enumerations

In [ ]:
emails_raw.groupby('ExtractedReleaseInPartOrFull').size()

In [ ]:
emails_raw.groupby('MetadataCaseNumber').size()

In [ ]:
emails_raw.groupby('MetadataDocumentClass').size()

## Restructuring the tables

The structure of the database does not respect the basic principles. We can see it if we draw the entity relationship diagram.

**TODO** insert here the picture of the ER provided.

The links between the tables should not be redundant because this leads to possible inconsistent states and errors.

**TODO** insert here the picture of the ER we will build.

### Choose relevant fields

Fortunately, `ExtractedDataSent` is redundant because it is contained in `MetadataDateSent` when it is not `null`. So we can simply drop it. This is great because it saves us from a lot of monkey work.

In [ ]:
print(emails_raw.MetadataDateSent.values[10])
print(emails_raw.ExtractedDateSent.values[10])

While testing manually random values, we conclude we will have to extract the content ourselves.

In [ ]:
# we have checked the for index 0 to 6, then random
idx = 1000

# The field `ExtractedReleaseInPartOrFull` seems to be useless
print(emails_raw.ExtractedReleaseInPartOrFull[idx])
print('***')
print(emails_raw.ExtractedBodyText[idx])
print('*******************')
print(emails_raw.RawText[idx])

To summarize, we keep temporarly (and will eventually rename):

* `Id` (id)
* `MetadataSubject` (subject)
* `SenderPersonId` (from)
* `MetadataTo` (to_alias)
* `ExtractedCc` (cc_alias)
* `MetadataDateSent` (sent)
* `MetadataDateReleased` (released)
* `MetadataCaseNumber` (case_number)
* `MetadataDocumentClass` (document_class)
* `RawText` (content)

In [6]:
# We only keep the fields which appear in the itemization above
emails = emails_raw[['Id', 'MetadataSubject', 'SenderPersonId', 'MetadataTo', 'ExtractedCc', 'MetadataDateSent', 'MetadataDateReleased', 'MetadataCaseNumber', 'MetadataDocumentClass', 'RawText']]

In [7]:
emails.head()

,Id,MetadataSubject,SenderPersonId,MetadataTo,ExtractedCc,MetadataDateSent,MetadataDateReleased,MetadataCaseNumber,MetadataDocumentClass,RawText
0,1,WOW,87.0,H,NaN,2012-09-12 04:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,NaN,H,NaN,2011-03-03 05:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,CHRIS STEVENS,32.0,;H,"Abedin, Huma",2012-09-12 04:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,CAIRO CONDEMNATION - FINAL,32.0,H,"Mitchell, Andrew B",2012-09-12 04:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,80.0,"Abedin, Huma",NaN,2011-03-11 05:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


### Recover dirty or missing values

We will proceed one field after the other because theese three columns require different strategies.

#### Recover `SenderPersonId`

We decided to read the content, line by line, and we define the first (non-empty) value.

#### Recover `MetadataTo`

Reading the value from the table `email_receivers` is not a good idea because the difference between destination and cc is not shown.

We find the `personId` through the alias provided in `MetadataTo`. Of couse, we need to preprocess it before fetching the person identifier. We will see it works pretty good: 7671 / 7690. If we include the `NaN`, we have 7671 / 7945.

For the `NaN` we use the same algorithm as the one used for extracting `SenderPersonId`.

#### Recover `ExtractedCC`

We use the table `email_receivers` and remove the row containing the value of `SenderPersonId` for each email identifier.

In [8]:
from extractor import Extractor
from process import Process

In [9]:
# SenderPersonId
counter_nan = 0
counter_recover = 0

for i in range(emails.shape[0]):
    if np.isnan(emails.SenderPersonId[i]):
        counter_nan += 1
        extracted = Extractor.sender_alias(emails.RawText[i])
        if extracted is not None:
            alias_found = Process.alias(extracted)
            person_id = aliases[aliases.Alias.str.strip() == alias_found].PersonId.values
            if len(person_id) == 1:
                emails.iat[i, 2] = person_id[0]
                counter_recover += 1
print("We manage to recover %d out of %d NaN for the sender_id." % (counter_recover, counter_nan))

We manage to recover 11 out of 157 NaN for the sender_id.


In [10]:
# MetadataTo
counter_not_nan = 0
counter_recover = 0

for i in range(emails.shape[0]):
    if not isinstance(emails.MetadataTo[i], str):
        continue
    counter_not_nan += 1
    alias = Process.alias(str(emails.MetadataTo[i]))
    person_id = aliases[aliases.Alias.str.strip() == alias].PersonId.values
    if len(person_id) == 1:
        emails.iat[i, 3] = person_id[0]
        counter_recover += 1
    else:
        print(emails.MetadataTo[i]) # could not match a person
print('***')
print("We manage to compute %d out of %d for the receiver_id." % (counter_recover, counter_not_nan))

michele.flournoy
Axelrod_D
Terry.Duffy
glantz.
rosemarie.howe ;H
cheryl.mills ;H
rrh.interiors
mh.interiors
H;preines
H;preines
Abedin, Huma; H
Abedin, Huma; H
Ki-moon, Ban
Sullivan, Jake; H
Etats-Unis D'Amerique
Etat-Unis D'Amerique
Duk-soo, Han
Duk-soo, Han
Betsy.Ebeling
***
We manage to compute 7671 out of 7690 for the receiver_id.


The `MetadataTo` field contains many `NaN`s and some elements (list above) could match a person. Hope is not lost because we can use extract this value from the `RawText` field.

In [11]:
emails.MetadataTo = pd.to_numeric(emails.MetadataTo, errors='coerce') # set NaN to string values

In [ ]:
# TODO perform the same algorithm from 'to' we did for 'from'

In [ ]:
# TODO the 'cc'.

## Text pre-processing

Now that we have extracted the relevant information, it is time to clean the content. Since the database we have contains less than 10,000 emails, we need to preprocess the data in an effective way. We need to remove common words and common sentences which appear almost on each email. This is necessary when running marchine learning algorithm on it to get better result.

In [12]:
emails['content'] = emails.RawText.map(Process.content)

In [16]:
# testing results manually
idx = 200
print(emails.content[idx])
print('***')
print(emails.RawText[idx])

b6 part b6 h hrod17 clintonemail com thursday january 5 2012 5 22 pm b6 h latest intel libyan conflicts leaders militias sid thanks always happy new year confidential january 5 2012 hrc libyan leadership militias source sources direct access libyan national transitional council well highest levels european governments western intelligence security services 1 last week december 2011 first week 2012 libya prime minister abdurrahim el keib president mustafa abdul jalil engaged series emergency planning meetings attempting deal specific issues threatening stability new national transitional council ntc government according extremely sensitive sources speaking strict confidence paramount among issues questions disarming rewarding regional militias bore majority fighting regime muammar al qaddafi well related issue finding ministers senior administrators new government acceptable revolutionary forces individuals noted four occasions beginning december 23 2011 groups angry militiamen came el 

## Augment Data

We would like to classify the email into (not excluding) categories like Middle East, Europe... But we do not have this information. We need to create it on our own. In order to build the data, we will use a system based on keywords. We will use this data as a based before training a neural network. We hope the neural network will discover some pattern other than the keywords.

In [20]:
emails['africa'] = emails.content.map(lambda content: Extractor.earth_area(content, "africa"))
emails['central_asia'] = emails.content.map(lambda content: Extractor.earth_area(content, "central_asia"))
emails['europe'] = emails.content.map(lambda content: Extractor.earth_area(content, "europe"))
emails['far_east'] = emails.content.map(lambda content: Extractor.earth_area(content, "far_east"))
emails['middle_east'] = emails.content.map(lambda content: Extractor.earth_area(content, "middle_east"))
emails['latino'] = emails.content.map(lambda content: Extractor.earth_area(content, "latino"))
emails['north_america'] = emails.content.map(lambda content: Extractor.earth_area(content, "north_america"))
emails['russia'] = emails.content.map(lambda content: Extractor.earth_area(content, "russia"))

In [21]:
emails.head()

,Id,MetadataSubject,SenderPersonId,MetadataTo,ExtractedCc,MetadataDateSent,MetadataDateReleased,MetadataCaseNumber,MetadataDocumentClass,RawText,content,africa,central_asia,europe,far_east,middle_east,latino,north_america,russia
0,1,WOW,87.0,80.0,NaN,2012-09-12 04:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,UNCLASSIFIED\nU.S. Department of State\nCase N...,wonderful strong moving statement boss please ...,False,False,False,False,False,False,False,False
1,2,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,194.0,80.0,NaN,2011-03-03 05:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,UNCLASSIFIED\nU.S. Department of State\nCase N...,part b6 attachments b6 thursday march 3 2011 9...,True,False,True,False,True,False,False,True
2,3,CHRIS STEVENS,32.0,80.0,"Abedin, Huma",2012-09-12 04:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,UNCLASSIFIED\nU.S. Department of State\nCase N...,part b6 thx original message soldiers thinking...,False,False,False,False,False,False,False,False
3,4,CAIRO CONDEMNATION - FINAL,32.0,80.0,"Mitchell, Andrew B",2012-09-12 04:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,UNCLASSIFIED\nU.S. Department of State\nCase N...,b5 b5 fyi shea peter ads aaron snipe spokesper...,False,False,False,False,False,False,False,False
4,5,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,80.0,81.0,NaN,2011-03-11 05:00:00,2015-05-22 04:00:00,F-2015-04841,HRC_Email_296,B6\nUNCLASSIFIED\nU.S. Department of State\nCa...,b6 part b6 attachments h hrod17 clintonemail c...,True,False,True,False,True,False,False,True


**Comment:** The result looks good but we surely need to do more on cleaning. For example, we the 'B6' almost everywhere.

## Network Analysis